# ML-7. Оптимизация гиперпараметров модели. Практика

In [24]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

## Подготовка

In [25]:
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

## 1.1 GridSearchCV + LogisticRegression

In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, # кросс-валидация (количество фолдов)
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 3.38 s
Wall time: 45.1 s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## 1.2 GridSearchCV + DecisionTreeClassifier

In [29]:
param_grid = {'min_samples_leaf': list(np.linspace(5, 100, 25, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 15, dtype=int)),
              'criterion':['entropy','gini']
              }
            
random_search_tree = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, # кросс-валидация (количество фолдов)
    n_jobs = -1
)  
%time random_search_tree.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search_tree.score(X_test, y_test)))
y_test_pred = random_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_tree.best_params_))

CPU times: total: 5.58 s
Wall time: 2min 30s
accuracy на тестовом наборе: 0.73
f1_score на тестовом наборе: 0.76
Наилучшие значения гиперпараметров: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 68}


## 2.1 RandomizedSearchCV + LogisticRegression

In [30]:
from sklearn.model_selection import RandomizedSearchCV

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_distributions = {'penalty': ['l2', 'none'] ,
                       'solver': ['lbfgs', 'sag'],
                       'C': list(np.linspace(0.01, 1, 10, dtype=float))
                       }
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, # кросс-валидация (количество фолдов)
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: total: 2.19 s
Wall time: 29.6 s
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.01}


c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## 2.2 RandomizedSearchCV + DecisionTreeClassifier

In [31]:
param_distributions = {'min_samples_leaf': list(np.linspace(5, 100, 25, dtype=int)),
                       'max_depth': list(np.linspace(1, 30, 15, dtype=int)),
                       'criterion':['entropy','gini']
                       }
            
random_search_tree = RandomizedSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5, # кросс-валидация (количество фолдов)
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search_tree.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search_tree.score(X_test, y_test)))
y_test_pred = random_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_tree.best_params_))

CPU times: total: 672 ms
Wall time: 11 s
accuracy на тестовом наборе: 0.73
f1_score на тестовом наборе: 0.76
Наилучшие значения гиперпараметров: {'min_samples_leaf': 68, 'max_depth': 3, 'criterion': 'gini'}


## 3.1 hyperopt + LogisticRegression

In [32]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

In [33]:
# зададим пространство поиска гиперпараметров
penalty_list = ['l2', 'none']
solver_list = ['lbfgs', 'sag']
c_list = list(np.linspace(0.01, 1, 10, dtype=float))
space = {'penalty': hp.choice('penalty', penalty_list) , # тип регуляризации
         'solver': hp.choice('solver', solver_list), # алгоритм оптимизации
         #'C': hp.quniform('C', 0.01, 1, 0.1)
         'C': hp.choice('C', c_list)
         } # уровень силы регурялизации

In [34]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state): 
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
              'C': float(params['C'])
              }    
    
    # используем эту комбинацию для построения модели
    #model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model = linear_model.LogisticRegression(**params, random_state=42, max_iter=50)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [35]:
# начинаем подбор гиперпараметров
%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          #rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
          rstate=np.random.default_rng(random_state),
          verbose=False
         )
print("Наилучшие значения гиперпараметров penalty = {}, solver = {}, C = {}".format(penalty_list[best['penalty']], solver_list[best['solver']], c_list[best['C']]))

CPU times: total: 0 ns
Wall time: 0 ns


c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\AL\AppData\Local\Programs\Python\Python38\l

Наилучшие значения гиперпараметров penalty = l2, solver = sag, C = 0.01


In [36]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(C=float(c_list[best['C']]),
    penalty=str(penalty_list[best['penalty']]),
    solver=str(solver_list[best['solver']]), 
    random_state=42, max_iter=50)

model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.82
accuracy на тестовом наборе: 0.75
f1_score на тестовом наборе: 0.78


c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## 3.2 hyperopt + DecisionTreeClassifier

In [37]:
# зададим пространство поиска гиперпараметров
criterion_list = ['entropy','gini']
space={'max_depth' : hp.quniform('max_depth', 1, 30, 2),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 100, 3),
       'criterion': hp.choice('criterion', criterion_list)
      }

In [38]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'max_depth': int(params['max_depth']), 
              'min_samples_leaf': int(params['min_samples_leaf']),
              'criterion': str(params['criterion'])
              }
  
    # используем эту комбинацию для построения модели
    model = tree.DecisionTreeClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [39]:
# начинаем подбор гиперпараметров
%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          #rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
          rstate=np.random.default_rng(random_state)
         )
print("Наилучшие значения гиперпараметров max_depth = {}, min_samples_leaf = {}, criterion = {}".format(best['max_depth'], best['min_samples_leaf'], criterion_list[best['criterion']]))

CPU times: total: 0 ns
Wall time: 0 ns
100%|██████████| 50/50 [00:45<00:00,  1.11trial/s, best loss: -0.7921147732439783]
Наилучшие значения гиперпараметров max_depth = 6.0, min_samples_leaf = 66.0, criterion = gini


In [40]:
# рассчитаем точность для тестовой выборки
model = tree.DecisionTreeClassifier(
    random_state=random_state, 
    min_samples_leaf=int(best['min_samples_leaf']),
    max_depth=int(best['max_depth']),
    criterion = criterion_list[best['criterion']]
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.81
accuracy на тестовом наборе: 0.73
f1_score на тестовом наборе: 0.76


## 4.1 optuna + LogisticRegression

In [41]:
import optuna

In [50]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
  C = trial.suggest_float('C', 0.01, 1, step=0.1)
  
  # создаем модель
  model = linear_model.LogisticRegression(C=C,
                                          penalty=penalty,
                                          solver=solver, 
                                          random_state=42, max_iter=50)
  # обучаем модель
  model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [51]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-10-31 01:12:46,415] A new study created in memory with name: LogisticRegression
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\distributions.py:668: UserWarning: The distribution is specified by [0.01, 1] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2022-10-31 01:12:48,287] Trial 0 finished with value: 0.7851506004638328 and parameters: {'penalty': 'l2'

CPU times: total: 50.2 s
Wall time: 1min 32s


In [52]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.11}


In [53]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params, random_state=random_state)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79


c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 4.2 optuna + DecisionTreeClassifier

In [54]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  criterion = trial.suggest_categorical('criterion', ['entropy','gini'])
  max_depth = trial.suggest_int('max_depth', 1, 30, 2)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 100, 3)
  
  # создаем модель
  model = tree.DecisionTreeClassifier(max_depth=max_depth,
                                      min_samples_leaf=min_samples_leaf,
                                      criterion=criterion, 
                                      random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [55]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="DecisionTreeClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-10-31 01:14:54,577] A new study created in memory with name: DecisionTreeClassifier
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\distributions.py:683: UserWarning: The distribution is specified by [1, 30] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 29].
  warnings.warn(
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\distributions.py:683: UserWarning: The distribution is specified by [5, 100] and step=3, but the range is not divisible by `step`. It will be replaced by [5, 98].
  warnings.warn(
[I 2022-10-31 01:14:55,558] Trial 0 finished with value: 0.7776521507139219 and parameters: {'criterion': 'gini', 'max_depth': 27, 'min_samples_leaf': 68}. Best is trial 0 with value: 0.7776521507139219.
c:\Users\AL\AppData\Local\Programs\Python\Python38\lib\site-packages\optuna\distributions.py:683: UserWarning: The distribution is specified by [1, 30] and step=2, but the range is not divisibl

CPU times: total: 15 s
Wall time: 48.4 s


In [56]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 98}


In [57]:
# рассчитаем точность для тестовой выборки
model = tree.DecisionTreeClassifier(**study.best_params, random_state=42)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.74
f1_score на тестовом наборе: 0.78
